In [1]:
!pip install  jupyter-dash
!pip install dash-bootstrap-components

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 10.4 MB/s 
     |████████████████████████████████| 357 kB 60.0 MB/s 
     |████████████████████████████████| 1.6 MB 50.0 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=b3c6d1d1347d7c4aabee56d30c89d09545836f618fa7c799f98e3db415c29482
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 8.2 MB/s 


In [2]:
# from getpass import getpass
# import os
# user = "asmaamohy"
# key = "109bae6fb0b390dcca9d9690f10d5b2e"

# if '.kaggle' not in os.listdir('/root'):
#     !mkdir ~/.kaggle
# !touch /root/.kaggle/kaggle.json
# !chmod 666 /root/.kaggle/kaggle.json
# with open('/root/.kaggle/kaggle.json', 'w') as f:
#     f.write('{"username":"%s","key":"%s"}' % (user, key))
# !chmod 600 /root/.kaggle/kaggle.json

# !kaggle competitions download -c m5-forecasting-accuracy

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# !unzip /content/m5-forecasting-accuracy.zip -d /content/drive/MyDrive/GP_iti

In [6]:
#imports 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()
from itertools import cycle

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import plotly.offline as py
import time
from tqdm import tqdm_notebook as tqdm
import os       
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

#Load Data
sales = pd.read_csv('/content/drive/MyDrive/GP_iti/sales_train_evaluation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/GP_iti/calendar.csv')
prices = pd.read_csv('/content/drive/MyDrive/GP_iti/sell_prices.csv')

In [7]:
##Downcast in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning:

`np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [8]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()

In [9]:
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left') 

In [ ]:
!pip install plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
group = sales.groupby(['state_id','store_id','cat_id','dept_id'],as_index=False)['item_id'].count().dropna()
group['USA'] = 'United States of America'
group.rename(columns={'state_id':'State','store_id':'Store','cat_id':'Category','dept_id':'Department','item_id':'Count'},inplace=True)
treefig = px.treemap(group[group['Count']!=0], path=['USA', 'State', 'Store', 'Category', 'Department'], values='Count',color='Count',
                     color_continuous_scale= px.colors.sequential.Sunset,
                  title='Walmart: Distribution of items')
treefig.update_layout(template='seaborn')
treefig.show()

In [ ]:
 # sales over time plot 
group = df.groupby(['year','date','state_id','store_id'], as_index=False)['sold'].sum().dropna()
fig_sales_time = go.Figure()
title = 'Items sold over time'
years = group.year.unique().tolist()
buttons = []
y=3
for state in group.state_id.unique().tolist():
    group_state = group[group['state_id']==state]
    for store in group_state.store_id.unique().tolist():
        group_state_store = group_state[group_state['store_id']==store]
        fig_sales_time.add_trace(go.Scatter(name=store, x=group_state_store['date'], y=group_state_store['sold'], showlegend=True, 
                                   yaxis='y'+str(y) if y!=1 else 'y'))
    y-=1

fig_sales_time.update_layout(
        xaxis=dict(
        #autorange=True,
        range = ['2011-01-29','2016-05-22'],
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(count=2,
                     label="2y",
                     step="year",
                     stepmode="backward"),
                dict(count=3,
                     label="3y",
                     step="year",
                     stepmode="backward"),
                dict(count=4,
                     label="4y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            autorange=True,
        ),
        type="date"
    ),
    yaxis=dict(
        anchor="x",
        autorange=True,
        domain=[0, 0.33],
        mirror=True,
        showline=True,
        side="left",
        tickfont={"size":10},
        tickmode="auto",
        ticks="",
        title='WI',
        titlefont={"size":20},
        type="linear",
        zeroline=False
    ),
    yaxis2=dict(
        anchor="x",
        autorange=True,
        domain=[0.33, 0.66],
        mirror=True,
        showline=True,
        side="left",
        tickfont={"size":10},
        tickmode="auto",
        ticks="",
        title = 'TX',
        titlefont={"size":20},
        type="linear",
        zeroline=False
    ),
    yaxis3=dict(
        anchor="x",
        autorange=True,
        domain=[0.66, 1],
        mirror=True,
        showline=True,
        side="left",
        tickfont={"size":10},
        tickmode="auto",
        ticks='',
        title="CA",
        titlefont={"size":20},
        type="linear",
        zeroline=False
    )
    )
fig_sales_time.update_layout(template='seaborn', title=title)
fig_sales_time.show()

In [ ]:
 group['state_id'].unique()

In [ ]:
state_list = ['California', 'Texas' , 'Wisconsin']

In [ ]:
state_inf={'California':'CA','Texas' :'TX','Wisconsin':'WI'}

In [ ]:
#group[group['state_id'] == 'CA']
state_dict = {'California':['CA_1', 'CA_2', 'CA_3', 'CA_4'] , 'Texas' :[ 'TX_1', 'TX_2', 'TX_3'], 'Wisconsin':['WI_1', 'WI_2' , 'WI_3']}

In [ ]:
store_list = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3','WI_1', 'WI_2' , 'WI_3']

In [ ]:
df['revenue'] = df['sold']*df['sell_price'].astype(np.float32)

In [ ]:
def introduce_nulls(df):
    idx = pd.date_range(df.date.dt.date.min(), df.date.dt.date.max())
    df = df.set_index('date')
    df = df.reindex(idx)
    df.reset_index(inplace=True)
    df.rename(columns={'index':'date'},inplace=True)
    return df


def plot_metric(df,state,store,metric):
    store_sales = df[(df['state_id']==state)&(df['store_id']==store)&(df['date']<='2016-05-22')]
    food_sales = store_sales[store_sales['cat_id']=='FOODS']
    store_sales = store_sales.groupby(['date','snap_'+state],as_index=False)['sold','revenue'].sum()
    snap_sales = store_sales[store_sales['snap_'+state]==1]
    non_snap_sales = store_sales[store_sales['snap_'+state]==0]
    food_sales = food_sales.groupby(['date','snap_'+state],as_index=False)['sold','revenue'].sum()
    snap_foods = food_sales[food_sales['snap_'+state]==1]
    non_snap_foods = food_sales[food_sales['snap_'+state]==0]
    non_snap_sales = introduce_nulls(non_snap_sales)
    snap_sales = introduce_nulls(snap_sales)
    non_snap_foods = introduce_nulls(non_snap_foods)
    snap_foods = introduce_nulls(snap_foods)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=non_snap_sales['date'],y=non_snap_sales[metric],
                           name='Total '+metric+'(Non-SNAP)'))
    fig.add_trace(go.Scatter(x=snap_sales['date'],y=snap_sales[metric],
                           name='Total '+metric+'(SNAP)'))
    fig.update_yaxes(title_text='Total items sold' if metric=='sold' else 'Total revenue($)')
    fig.update_layout(template='seaborn',title=store)
    fig.update_layout(
        xaxis=dict(
        #autorange=True,
        range = ['2011-01-29','2016-05-22'],
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(count=2,
                     label="2y",
                     step="year",
                     stepmode="backward"),
                dict(count=3,
                     label="3y",
                     step="year",
                     stepmode="backward"),
                dict(count=4,
                     label="4y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            autorange=True,
        ),
        type="date"
    ))
    return fig

In [ ]:
fig = plot_metric(df,'CA','CA_1','sold')
fig.show()

In [ ]:
#Load Data after modeling 
predictions = pd.read_csv('/content/drive/MyDrive/GP_iti/submission.csv')
predictions=predictions[~predictions['id'].str.contains('_evaluation')]
x=pd.date_range(start="2016-05-23",end="2016-06-19")
predictions=predictions.set_index('id').T
predictions['date']=x
l_products= [c.replace('_validation','') for c in list(predictions.columns)]
dict_name={k:v for k,v in zip(list(predictions.columns),l_products)}
predictions= predictions.rename(columns=dict_name)
predictions= predictions.set_index('date')
px.line(predictions['HOBBIES_1_001_CA_1'],markers=True,title= 'HOBBIES_1_001_CA_1'+' sales').update_layout(showlegend=False,title_x=0.5)

In [ ]:
import dash #main package 
import dash_bootstrap_components as dbc #for dash styling 
from jupyter_dash import JupyterDash
from dash import html,Input,Output,State,dcc


md_docs = ''' 
#####   PROJECT EXPLANATION 
---
This project used The M5 dataset, generously made available by Walmart, involves the unit sales of various products sold 
in the USA, organized in the form of grouped time series. More specifically, the dataset involves the unit 
sales of 3,049 products, classified in 3 product categories (Hobbies, Foods, and Household) and 7 product 
departments, in which the above-mentioned categories are disaggregated. The products are sold across 
ten stores, located in three States (CA, TX, and WI), The historical data range from 2011-01-29 to 2016-06-19.
 
[Click here to get the Dataset](https://www.kaggle.com/competitions/m5-forecasting-accuracy/data)
 
This Project was created to aim these following objectives : 
- Exploration  **Time Series Data**.
- Predict 28 days ahead point.
- Create **Business Intelligence Dashboard** .given period
    
##### PROJECT WORKFLOW 
___
1. Data Cleaning (notebook : )
2. Data Exploration 
3. Creating Forecast Model 
4. Model Evaluation 
5. Creating Dashboard
 
#### Package / Tech Stack Used 
---
- Dashboard Development : 
    1. Jupyter Dash
    2. Dash Bootstrap Component 
    3. Dash Extention 
    4. Plotly 
- Time Series Model Development 
    1. statsmodels
    2. Pandas 
    3. Numpy
    
#### FUTURE PLANS 
---
- Time Series Model Development 
    1. Use Deep Learning, Supervised ML, 
    2. Pandas 
    3. Numpy
    
'''
app = JupyterDash(external_stylesheets=[dbc.themes.CERULEAN])

#basic styling

stats_basic_container_style = {'background-color':'#378dfc','color':'#fcfaff','text-align': 'center'}
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "padding": "2rem 1rem",
    "background-color": "#3083ff",
}

CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
    'background-color' : '#f0f3f7'

}
#creating sidebar to access multiple apps
sidebar = html.Div(
    [
        html.H4("Choose Menu Below : ", style={'color':'#fcfaff'}),
        dbc.CardBody([
            dbc.Nav(
            [   dbc.NavLink("Project Explanation", href="/", active="exact"),
                dbc.NavLink("Statistics", href="/stats", active="exact"),
                dbc.NavLink("Predictive Analytics", href="/predict", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],style={'background-color' : '#f0f3f7'} ),
    
    ],style=SIDEBAR_STYLE)

#render selected navbar
content = html.Div(id="page-content", style=CONTENT_STYLE)



app.layout = html.Div([dcc.Location(id="url"), sidebar, content])

#callback to change content based of navbar clicked
@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page(pathname) : 
    if pathname == '/' : 
        content = dcc.Markdown(md_docs)
        content = dbc.Container(
        children = [
            html.Br(), 
            dbc.Row(className='dashboard_picture',
                children = [html.Img(src='https://raw.githubusercontent.com/fakhrirobi/forecast_passenger_BI/main/assets/path-digital-tR0jvlsmCuQ-unsplash.jpg',alt='Dashboard Picture', 
                         style={

                              'height':700,
                               "display": "block",
                                "margin-left": "auto",
                                "margin-right": "auto"
                         })]),
        html.Br(),
        dcc.Markdown(md_docs),
        html.H4('Display Tree Map for distribution of 3049 items across different aggregation levels.',),
        html.Div([dcc.Graph(figure= treefig,id="MapGraph")]),
            
        
        ])
        return content
    elif pathname == '/stats' : 
        return html.Div(dbc.Card([
            html.H2('Number of items sold over time across all the stores',className='card-title text-center'),
            html.Div([dcc.Graph(figure= fig_sales_time,id="SelesGraph")]),

            html.H2('The Sales and Revenue of all the stores across States',className='card-title text-center'),
            # Selection 
           dbc.CardBody([
                               html.H6("Select State"),
                               dcc.RadioItems(id="state_select",options = state_list ,labelStyle ={'cursor': 'pointer','margin-left':'20px'},value ='California')
      ]),
                    dbc.CardBody([
                               html.H6("Select Store"),
                               dcc.Dropdown(id="store_select"),
      ]),
            #plots 
                
                html.Div(dbc.Row([
                dbc.Col([
                    html.H6("Revenue Distirbution"),
                    dcc.Graph(figure= {},id="RevenueGraph") 
                ]),
                dbc.Col([
                    html.H6("Sales Distirbution"),
                    dcc.Graph(figure= {},id="SalesGraph")
                ]) ])),

                                  
                                  
        ]))
    elif pathname == '/predict' : 
        return html.Div(dbc.Card([
            html.H2('Forcast 28 Days',className='card-title text-center'),
            #Bans
             dbc.Row(
                    [dbc.Col(dbc.Card(
                                        [
                                            html.H4(id='Sale Error'+" 🔻 ",className='card-title text-center'), 
                                            html.H6(id='0.5$',className='card-title text-center')
                                        ]
                        ,style=stats_basic_container_style)), 
                     dbc.Col(dbc.Card(
                                        [ 
                                            html.H6(id='Competition Score'+'🌟',className='card-title text-center'), 
                                            html.H4(id='0.4',className='card-title text-center')
                                        ]
                         ,style=stats_basic_container_style)), ]
                     
                ),
            # Selection 
            dbc.CardBody([
                               html.H4('Select Product'),
                               dcc.Dropdown(id="product_select",options = l_products),
      ]),
                html.Div([dcc.Graph(figure= {},id="ForcastGraph")]),
                     
                                  
        ]))
    else : 
        return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )


#Selection callback
@app.callback(
    Output(component_id="store_select", component_property='options'),
    [Input(component_id='state_select', component_property="value"),],
    )
def select_store(state):
    # global temp_state
    # temp_state = state
    return  state_dict[state]

# Display Figures
@app.callback(
    Output(component_id="RevenueGraph",component_property="figure"),
    Output(component_id="SalesGraph",component_property="figure"), 
    [Input(component_id="store_select",component_property="value"),
     Input(component_id='state_select', component_property="value"),],
    )
def dispaly(store,state):
    return  plot_metric(df,state_inf[state],store,'revenue'), plot_metric(df,state_inf[state],store,'sold')


@app.callback(
    Output(component_id="ForcastGraph",component_property="figure"), 
    [Input(component_id="product_select",component_property="value"),],
    )
def forcast(product):
    return  px.line(predictions[product],markers=True,title= product +' sales').update_layout(showlegend=False,title_x=0.5)
app.run_server(debug=True)